## Training using ConvNeXT-S

In [1]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ['TORCH_USE_CUDA_DSA'] = "1"

In [2]:
import os
import torch
from PIL import Image
import numpy as np
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn, optim

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
torch.__version__

'2.9.1+cu126'

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [6]:
# models.resnet152(weights='IMAGENET1K_V2')
# models.convnext_small(weights='IMAGENET1K_V1')
# models.efficientnet_v2_s(weights='IMAGENET1K_V1')
# models.mobilenet_v2(weights='IMAGENET1K_V2')

In [7]:
class FoodDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(data_dir))
        self.class_to_idx = {cls: i for i,cls in enumerate(self.classes)}

        for label_name in self.classes:
            label_dir = os.path.join(data_dir, label_name)
            for img_name in os.listdir(label_dir):
                self.image_paths.append(os.path.join(label_dir, img_name))
                self.labels.append(self.class_to_idx[label_name])

    
    def __len__(self):
        return len(self.image_paths)

    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [8]:
input_size = 224

# ImageNet Normalization values
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Simple transforms - just resize and normalize
train_transforms = transforms.Compose([
    transforms.Resize((230, 230)),
    transforms.RandomRotation(15),           # Rotate up to 10 degrees
    transforms.RandomResizedCrop(224, scale=(0.9, 1.0)),  # Zoom
    transforms.RandomHorizontalFlip(),       # Horizontal flip
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

val_transforms = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [9]:
train_dataset = FoodDataset(
    data_dir='./indfood-data/train',
    transform=train_transforms
)

val_dataset = FoodDataset(
    data_dir='./indfood-data/val',
    transform=val_transforms
)


In [10]:
total_train_len = len(train_dataset)
total_val_len = len(val_dataset)

# Define the length of the smaller dataset you want to use
subset_train_len = 25000
subset_val_len = 6000

# Define the lengths for the split: [desired_length, remaining_length]
train_lengths = [subset_train_len, total_train_len - subset_train_len]
val_lengths = [subset_val_len, total_val_len - subset_val_len]

# Randomly split the original dataset into two new datasets, You get a list of datasets; take the first one [0]
smaller_train_dataset = random_split(train_dataset, train_lengths)[0]
smaller_val_dataset = random_split(val_dataset, val_lengths)[0]


In [11]:
train_loader = DataLoader(smaller_train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(smaller_val_dataset, batch_size=32, shuffle=False)

In [12]:
len(smaller_train_dataset), len(smaller_val_dataset)

(25000, 6000)

In [13]:
classes = list(train_dataset.class_to_idx)
classes

['aloo_gobi',
 'aloo_matar',
 'aloo_methi',
 'aloo_paratha',
 'aloo_shimla_mirch',
 'aloo_tikki',
 'amritsari_kulcha',
 'anda_curry',
 'ariselu',
 'balushahi',
 'banana_chips',
 'bandar_laddu',
 'basundi',
 'besan_laddu',
 'bhindi_masala',
 'biryani',
 'boondi',
 'boondi_laddu',
 'butter_chicken',
 'chaas',
 'chak_hao_kheer',
 'cham_cham',
 'chana_masala',
 'chapati',
 'chicken_pizza',
 'chicken_razala',
 'chicken_tikka',
 'chicken_tikka_masala',
 'chicken_wings',
 'chikki',
 'chivda',
 'chole_bhature',
 'daal_baati_churma',
 'daal_puri',
 'dabeli',
 'dal_khichdi',
 'dal_makhani',
 'dal_tadka',
 'dharwad_pedha',
 'dhokla',
 'double_ka_meetha',
 'dum_aloo',
 'falooda',
 'fish_curry',
 'gajar_ka_halwa',
 'garlic_bread',
 'gavvalu',
 'ghevar',
 'grilled_sandwich',
 'gujhia',
 'gulab_jamun',
 'hara_bhara_kabab',
 'idiyappam',
 'idli',
 'imarti',
 'jalebi',
 'kachori',
 'kadai_paneer',
 'kadhi_pakoda',
 'kaju_katli',
 'kakinada_khaja',
 'kalakand',
 'karela_bharta',
 'khakhra',
 'kheer',
 '

In [14]:
num_classes = len(classes)
num_classes

131

In [15]:
class FoodClassifierConvNext(nn.Module):
    def __init__(self, num_classes=131):
        super(FoodClassifierConvNext, self).__init__()

        # load pre-trained ConvNeXT-S
        self.base_model = models.convnext_small(weights='IMAGENET1K_V1')

        # freeze base model parameters
        for param in self.base_model.parameters():
            param.requires_grad = False

        # remove original classifier
        self.base_model.classifier = nn.Identity()

        # add custom layers
        self.global_avg_pooling = nn.AdaptiveAvgPool2d((1,1))
        self.output_layer = nn.Linear(768, num_classes)

    def forward(self, x):
        x = self.base_model.features(x)
        x = self.global_avg_pooling(x)
        x = torch.flatten(x, 1)
        x = self.output_layer(x)
        return x
        

In [16]:
model = FoodClassifierConvNext(num_classes=131)
model.to(device)

FoodClassifierConvNext(
  (base_model): ConvNeXt(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
        (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
      )
      (1): Sequential(
        (0): CNBlock(
          (block): Sequential(
            (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
            (1): Permute()
            (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
            (3): Linear(in_features=96, out_features=384, bias=True)
            (4): GELU(approximate='none')
            (5): Linear(in_features=384, out_features=96, bias=True)
            (6): Permute()
          )
          (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        )
        (1): CNBlock(
          (block): Sequential(
            (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
            (1): Permute()
            (2): L

In [17]:
criterion = nn.CrossEntropyLoss()

In [16]:
# tuning the learning rate
def make_model(learning_rate=0.01):
    model = FoodClassifierConvNext(num_classes=131)
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    return model, optimizer


In [18]:
def train_and_evaluate(model, optimizer, train_loader, val_loader, criterion, num_epochs, device):
    best_val_accuracy = 0.0  # Initialize variable to track the best validation accuracy

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            # backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_acc = correct / total

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_acc = val_correct / val_total

        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            checkpoint_path = f'food_cnext_v33_{epoch+1:02d}_{val_acc:.3f}.pth'
            torch.save(model.state_dict(), checkpoint_path)
            print(f'Checkpoint saved: {checkpoint_path}')

### Tuning the Learning Rate

In [34]:
num_epochs = 10
for lr in [0.1, 0.01, 0.001, 0.0001]:
    print("learning rate =", lr)
    model, optimizer = make_model(lr)
    train_and_evaluate(model, optimizer, train_loader, val_loader, criterion, num_epochs, device)
    print()
    print()

learning rate = 0.1
Epoch 1/10
  Train Loss: 3.5548, Train Acc: 0.4974
  Val Loss: 2.5158, Val Acc: 0.6572
Checkpoint saved: food_cnext_v31_01_0.657.pth
Epoch 2/10
  Train Loss: 2.8298, Train Acc: 0.6195
  Val Loss: 2.9130, Val Acc: 0.6628
Checkpoint saved: food_cnext_v31_02_0.663.pth
Epoch 3/10
  Train Loss: 2.6588, Train Acc: 0.6645
  Val Loss: 3.6665, Val Acc: 0.6364
Epoch 4/10
  Train Loss: 2.4327, Train Acc: 0.6985
  Val Loss: 3.0065, Val Acc: 0.6900
Checkpoint saved: food_cnext_v31_04_0.690.pth
Epoch 5/10
  Train Loss: 2.3515, Train Acc: 0.7118
  Val Loss: 3.0729, Val Acc: 0.7052
Checkpoint saved: food_cnext_v31_05_0.705.pth
Epoch 6/10
  Train Loss: 2.4107, Train Acc: 0.7168
  Val Loss: 3.1426, Val Acc: 0.7264
Checkpoint saved: food_cnext_v31_06_0.726.pth
Epoch 7/10
  Train Loss: 2.3150, Train Acc: 0.7318
  Val Loss: 3.4725, Val Acc: 0.7144
Epoch 8/10
  Train Loss: 2.3249, Train Acc: 0.7434
  Val Loss: 3.6227, Val Acc: 0.7168
Epoch 9/10
  Train Loss: 2.3313, Train Acc: 0.7475
  V

In [37]:
# best lrs : 0.01. 0.001

### Tuning the Number of Inner Layers to be added

In [18]:
class FoodClassifierConvNext(nn.Module):
    def __init__(self, size_inner=100, num_classes=131):
        super(FoodClassifierConvNext, self).__init__()

        # load pre-trained convnext-s
        self.base_model = models.convnext_small(weights='IMAGENET1K_V1')

        # freeze base model parameters
        for param in self.base_model.parameters():
            param.requires_grad = False

        # remove original classifier
        self.base_model.classifier = nn.Identity()

        # add custom layers
        self.global_avg_pooling = nn.AdaptiveAvgPool2d((1,1))
        # add inner layers
        self.inner = nn.Linear(768, size_inner)  # New inner layer
        self.relu = nn.ReLU()
        self.output_layer = nn.Linear(size_inner, num_classes)

    def forward(self, x):
        x = self.base_model.features(x)
        x = self.global_avg_pooling(x)
        x = torch.flatten(x, 1)
        x = self.inner(x)
        x = self.relu(x)
        x = self.output_layer(x)
        return x


def make_model(
        learning_rate=0.01,
        size_inner=100
):
    model = FoodClassifierConvNext(
        num_classes=131,
        size_inner=size_inner
    )
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    return model, optimizer


In [19]:
num_epochs = 10
for size_inner in [1000, 500, 200, 100]:
    print(f"size_inner : {size_inner}")
    model, optimizer = make_model(
        learning_rate=0.01,
        size_inner=size_inner
    )
    
    train_and_evaluate(model, optimizer, train_loader, val_loader, criterion, num_epochs, device)
    print()

size_inner : 1000
Epoch 1/10
  Train Loss: 1.9910, Train Acc: 0.5055
  Val Loss: 1.3256, Val Acc: 0.6624
Checkpoint saved: food_cnext_v32_01_0.662.pth
Epoch 2/10
  Train Loss: 1.3054, Train Acc: 0.6399
  Val Loss: 1.3282, Val Acc: 0.6616
Epoch 3/10
  Train Loss: 1.1289, Train Acc: 0.6850
  Val Loss: 1.2549, Val Acc: 0.6972
Checkpoint saved: food_cnext_v32_03_0.697.pth
Epoch 4/10
  Train Loss: 1.0295, Train Acc: 0.7052
  Val Loss: 1.2157, Val Acc: 0.7160
Checkpoint saved: food_cnext_v32_04_0.716.pth
Epoch 5/10
  Train Loss: 0.9638, Train Acc: 0.7195
  Val Loss: 1.2691, Val Acc: 0.7132
Epoch 6/10
  Train Loss: 0.8790, Train Acc: 0.7426
  Val Loss: 1.3636, Val Acc: 0.7056
Epoch 7/10
  Train Loss: 0.8695, Train Acc: 0.7507
  Val Loss: 1.3369, Val Acc: 0.7140
Epoch 8/10
  Train Loss: 0.8102, Train Acc: 0.7613
  Val Loss: 1.3878, Val Acc: 0.7152
Epoch 9/10
  Train Loss: 0.7975, Train Acc: 0.7649
  Val Loss: 1.3572, Val Acc: 0.7260
Checkpoint saved: food_cnext_v32_09_0.726.pth
Epoch 10/10
  T

KeyboardInterrupt: 

In [20]:
# best inner layers size : 500

### Tuning the Drop Rate

In [19]:
class FoodClassifierConvNext(nn.Module):
    def __init__(self, size_inner=100, droprate=0.2, num_classes=131):
        super(FoodClassifierConvNext, self).__init__()

        # load pre-trained ConvNeXT-S
        self.base_model = models.convnext_small(weights='IMAGENET1K_V1')

        # freeze base model parameters
        for param in self.base_model.parameters():
            param.requires_grad = False

        # remove original classifier
        self.base_model.classifier = nn.Identity()

        # add custom layers
        self.global_avg_pooling = nn.AdaptiveAvgPool2d((1,1))
        # add inner layers
        self.inner = nn.Linear(768, size_inner)  # New inner layer
        self.relu = nn.ReLU()
        # add dropout
        self.dropout = nn.Dropout(droprate)
        self.output_layer = nn.Linear(size_inner, num_classes)

    def forward(self, x):
        x = self.base_model.features(x)
        x = self.global_avg_pooling(x)
        x = torch.flatten(x, 1)
        x = self.inner(x)
        x = self.relu(x)
        x = self.dropout(x)  # apply dropout
        x = self.output_layer(x)
        return x


def make_model(
        learning_rate=0.001,
        size_inner=500,
        droprate=0.2
):
    model = FoodClassifierConvNext(
        num_classes=131,
        size_inner=size_inner,
        droprate=droprate
    )
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    return model, optimizer


In [21]:
num_epochs = 50
for drop_rate in [0.3,]: #0.2, 0.1]:
    print(f"drop_rate : {drop_rate}")
    model, optimizer = make_model(
        learning_rate=0.001,
        size_inner=500,
        droprate=drop_rate
    )
    
    train_and_evaluate(model, optimizer, train_loader, val_loader, criterion, num_epochs, device)
    print()

drop_rate : 0.3
Epoch 1/50
  Train Loss: 1.9887, Train Acc: 0.5201
  Val Loss: 1.1347, Val Acc: 0.7007
Checkpoint saved: food_cnext_v33_01_0.701.pth
Epoch 2/50
  Train Loss: 1.1725, Train Acc: 0.6798
  Val Loss: 0.9372, Val Acc: 0.7478
Checkpoint saved: food_cnext_v33_02_0.748.pth
Epoch 3/50
  Train Loss: 1.0020, Train Acc: 0.7214
  Val Loss: 0.8324, Val Acc: 0.7627
Checkpoint saved: food_cnext_v33_03_0.763.pth
Epoch 4/50
  Train Loss: 0.8982, Train Acc: 0.7435
  Val Loss: 0.7654, Val Acc: 0.7863
Checkpoint saved: food_cnext_v33_04_0.786.pth
Epoch 5/50
  Train Loss: 0.8369, Train Acc: 0.7564
  Val Loss: 0.7529, Val Acc: 0.7842
Epoch 6/50
  Train Loss: 0.7872, Train Acc: 0.7700
  Val Loss: 0.7158, Val Acc: 0.7955
Checkpoint saved: food_cnext_v33_06_0.795.pth
Epoch 7/50
  Train Loss: 0.7430, Train Acc: 0.7793
  Val Loss: 0.7024, Val Acc: 0.7993
Checkpoint saved: food_cnext_v33_07_0.799.pth
Epoch 8/50
  Train Loss: 0.7135, Train Acc: 0.7903
  Val Loss: 0.6944, Val Acc: 0.8067
Checkpoint s

In [22]:
# best drop rate : 0.3

In [ ]:
# num_epochs = 50
# model, optimizer = make_model(
#     learning_rate=0.001,
#     size_inner=1000,
#     droprate=0.4
# )

# train_and_evaluate(model, optimizer, train_loader, val_loader, criterion, num_epochs, device)

### Model Summary

In [23]:
from torchsummary import summary
summary(model, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 56, 56]           4,704
       LayerNorm2d-2           [-1, 96, 56, 56]             192
            Conv2d-3           [-1, 96, 56, 56]           4,800
           Permute-4           [-1, 56, 56, 96]               0
         LayerNorm-5           [-1, 56, 56, 96]             192
            Linear-6          [-1, 56, 56, 384]          37,248
              GELU-7          [-1, 56, 56, 384]               0
            Linear-8           [-1, 56, 56, 96]          36,960
           Permute-9           [-1, 96, 56, 56]               0
  StochasticDepth-10           [-1, 96, 56, 56]               0
          CNBlock-11           [-1, 96, 56, 56]               0
           Conv2d-12           [-1, 96, 56, 56]           4,800
          Permute-13           [-1, 56, 56, 96]               0
        LayerNorm-14           [-1, 56,

### Exporting to ONNX for serverles model deployment

In [24]:

dummy_input = torch.randn(1, 3, 224, 224).to(device)

# Export to ONNX
onnx_path = "food_classifier_convnexts_v2.onnx"

torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    verbose=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={
        'input': {0: 'batch_size'},
        'output': {0: 'batch_size'}
    },
)

[torch.onnx] Obtain model graph for `FoodClassifierConvNext([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `FoodClassifierConvNext([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅


Skipping constant folding for op SequenceEmpty with multiple outputs.
Skipping constant folding for op SequenceEmpty with multiple outputs.


Applied 2 of general pattern rewrite rules.


ONNXProgram(
    model=
        <
            ir_version=10,
            opset_imports={'': 20},
            producer_name='pytorch',
            producer_version='2.9.1+cu126',
            domain=None,
            model_version=None,
        >
        graph(
            name=main_graph,
            inputs=(
                %"input"<FLOAT,[s77,3,224,224]>
            ),
            outputs=(
                %"output"<FLOAT,[s77,131]>
            ),
            initializers=(
                %"base_model.features.0.0.weight"<FLOAT,[96,3,4,4]>{TorchTensor(...)},
                %"base_model.features.0.0.bias"<FLOAT,[96]>{TorchTensor(...)},
                %"base_model.features.0.1.weight"<FLOAT,[96]>{TorchTensor(...)},
                %"base_model.features.0.1.bias"<FLOAT,[96]>{TorchTensor(...)},
                %"base_model.features.1.0.layer_scale"<FLOAT,[96,1,1]>{TorchTensor(...)},
                %"base_model.features.1.0.block.0.weight"<FLOAT,[96,1,7,7]>{TorchTensor(...)},
         

In [ ]:
# ## load model from a checkpoint
# path = './clothing_v4_23_0.830.pth'
# model = ClothingClassifierMobileNet(size_inner=100, droprate=0.2, num_classes=10)
# model.load_state_dict(torch.load(path))
# model.to(device)
# model.eval();

In [ ]:
# x = val_transforms(img)
# batch_t = torch.unsqueeze(x, 0).to(device)

# with torch.no_grad():
#     output = model(batch_t)

In [ ]:
# dict(zip(classes, output[0].to('cpu')))